In [ ]:
%pip install llama-index-embeddings-azure-openai
%pip install llama-index-llms-azure-openai
%pip install langchain_openai
%pip install llama-index
%pip install langchain
%pip install PyPDF2
%pip install -q cassio datasets langchain openai tiktoken
%pip install pypdf
%pip install faiss-cpu
%pip install langchain
%pip install langchainhub
%pip install openai
%pip install --upgrade --quiet  langchain langchain-openai
%pip install huggingface-hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.7/164.7 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai faiss-cpu

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain.agents.initialize import initialize_agent
from langchain.agents import Tool, AgentExecutor, load_tools
from langchain.agents.react.base import DocstoreExplorer
from langchain_community.document_loaders import PyPDFDirectoryLoader
import os
from langchain.tools import BaseTool
from langchain import docstore
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
import faiss
from langchain import hub
from langchain.chat_models import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain.agents import AgentExecutor, create_openai_tools_agent


os.environ["OPENAI_API_KEY"] = ""

embeddings = AzureOpenAIEmbeddings(
    azure_deployment="textembeddingad002dep2",
    openai_api_version="2023-07-01-preview",
)

llm_azure = AzureChatOpenAI(openai_api_version="2023-07-01-preview",
                            azure_deployment='gpt35turbo16kdep2',
                            temperature=0
                            )

documents = PyPDFDirectoryLoader("data").load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(texts, embeddings,)

retriever = db.as_retriever()

tool = create_retriever_tool(
    retriever,
    "search_health_cancer",
    "Searches and returns Conditions and Surgeries covered in Life Heart Shield Plan",
)
tools = [tool]

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

agent = create_openai_tools_agent(llm_azure, tools, prompt)
# agent = create_react_agent(llm_azure, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)
#print(agent_executor)

result = agent_executor.invoke(
    {
        "input": "What are the different Conditions and Surgeries covered in Life Heart Shield Plan?"
    }
)

result["output"]

'The Life Heart Shield Plan covers the following Conditions and Surgeries:\n\nGroup A: High severity Covered Conditions eligible for 100% of Sum Insured:\n- Myocardial Infarction (First Heart Attack – of Specified Severity)\n- Open Chest CABG\n- Open Heart Replacement or Repair of Heart Valves\n- Major Surgery of Aorta\n- Heart Transplant\n- Cardiomyopathy\n- Primary (Idiopathic) Pulmonary Hypertension\n\nGroup B: Moderate severity Covered conditions eligible for 50% of Sum Insured:\n- Balloon Valvotomy or Valvuloplasty\n- Surgery to place Ventricular Assist Devices or Total Artificial Hearts\n- Implantable Cardioverter Defibrillator (ICD)\n- Carotid Artery Surgery\n\nGroup C: Mild severity Covered Conditions eligible for 25% of Sum Insured:\n- Pericardectomy\n- Minimally Invasive Surgery of Aorta\n- Angioplasty\n- Infective Endocarditis\n\nPlease note that this information is based on the policy document for the Life Heart Shield Plan.'